<a href="https://colab.research.google.com/github/AmirHesamKamalpour/Genetic-Algorithm-solving-0-1-Knapsack/blob/main/GeneticAlgorithmKnapsack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import math
import concurrent.futures

In [ ]:
def BuildRandomPopulation(pSize : int, N: int):
  population = np.random.randint(2, size=(pSize, N))
  return population

In [ ]:
def Fitness(Chromosome, W, weights, profits):
  fitness = 0
  occupiedWeight = 0

  for index, gene in enumerate(Chromosome):
    occupiedWeight += gene * weights[index]
    fitness += gene * profits[index]

  return fitness if occupiedWeight <= W else 0

In [ ]:
def AddEpsilon(fitness_values, epsilon=1e-6):
    adjusted_fitness = fitness_values + epsilon
    return adjusted_fitness

In [ ]:
def SelectParents(population, W, weights, profits, numNewPopulation): # Roulette wheel selection is applied
  fitnesses = []
  parents = np.zeros(population.shape)
  for row in range(population.shape[0]):
    fitnesses.append(Fitness(population[row], W, weights, profits))

  fitnesses = AddEpsilon(np.array(fitnesses))
  cumulativeProbs = np.cumsum(fitnesses) / fitnesses.sum()
  randomNumbers = np.random.rand(numNewPopulation)

  for index, randomNumber in enumerate(randomNumbers):
    selectedindex = np.searchsorted(cumulativeProbs, randomNumber)
    parents[index] = population[selectedindex]

  return parents

In [ ]:
def TwoPointCrossOver(parents, crossOverChance=0.85):
  offsprings = np.zeros(parents.shape)
  crossOverPoints = np.random.randint(1, parents.shape[1], size=(2,))
  crossOverPoints.sort()
  i = 0
  while i + 2 <= parents.shape[0]:
    if np.random.random() < crossOverChance:
      offsprings[i] = np.concatenate((parents[i][:crossOverPoints[0]], parents[i + 1][crossOverPoints[0]:crossOverPoints[1]], parents[i][crossOverPoints[1]:]))
      offsprings[i + 1] = np.concatenate((parents[i + 1][:crossOverPoints[0]], parents[i][crossOverPoints[0]:crossOverPoints[1]], parents[i + 1][crossOverPoints[1]:]))
    else:
      offsprings[i] = parents[i]
      offsprings[i + 1] = parents[i + 1]
    i += 2
  return offsprings

In [ ]:
def SinglePointCrossOver(parents, crossOverChance = .85):
  offsprings = np.zeros(parents.shape)
  crossOverPoint = np.random.randint(1, parents.shape[1])
  i = 0
  while i+2 <= parents.shape[0]:

    if np.random.random() < crossOverChance:

      offsprings[i] = np.concatenate((parents[i][:crossOverPoint], parents[i+1][crossOverPoint:]))
      offsprings[i+1] = np.concatenate((parents[i+1][:crossOverPoint], parents[i][crossOverPoint:]))

    else:

      offsprings[i] = parents[i]
      offsprings[i+1] = parents[i+1]

    i += 2

  return offsprings

In [ ]:
def Mutation(offsprings, mutationChance = .01):
  mutatedOffsprings = offsprings.copy()
  mutationChances = np.random.random(offsprings.shape)
  mutatedOffsprings[mutationChances < mutationChance] = np.logical_not(mutatedOffsprings[mutationChances < mutationChance]).astype(int)
  return mutatedOffsprings

In [ ]:
def RunProgram(iterations, num, weightLimit ,weights, profits, identicalFitLimit = .80, populationSize = 500):

  a = BuildRandomPopulation(populationSize, num)
  b = SelectParents(a, weightLimit, weights, profits, populationSize)
  c = TwoPointCrossOver(b)
  d = Mutation(c)

  iterationNumber = 0
  for i in range(iterations ** 2):
    iterationNumber +=1
    unq, cnt = np.unique(d, axis=0, return_counts=True)
    if (cnt.max() / populationSize) >= identicalFitLimit and i >= iterations:
      print("Number of generations: ",iterationNumber)
      return unq[cnt.argmax()]
    b = SelectParents(d, weightLimit, weights, profits, populationSize)
    c = TwoPointCrossOver(b)
    d = Mutation(c)

  unq, cnt = np.unique(d, axis=0, return_counts=True)
  print("Number of generations: ",iterationNumber)
  return unq[cnt.argmax()]

In [ ]:
N = int(input("Enter the number of items: "))
W = int(input("Enter the maximum weight limit: "))
profits = [int(x) for x in input("Enter the values of items: ").split()]
weights = [int(x) for x in input("Enter the weights of items: ").split()]

Enter the number of items: 10
Enter the maximum weight limit: 165
Enter the values of items: 92 57 49 68 60 43 67 84 87 72
Enter the weights of items: 23 31 29 44 53 38 63 85 89 82


In [ ]:
def RunProgramOnThread(threadsNum):

  with concurrent.futures.ThreadPoolExecutor() as executor:
    chromo = np.zeros((threadsNum,N))
    results = [executor.submit(RunProgram, int(math.pow(2,N))) for _ in range(threadsNum)]
    for index, f in enumerate(concurrent.futures.as_completed(results)):
      chromo[index] = f.result()
  unq, cnt = np.unique(chromo, axis=0, return_counts=True)
  print(unq[cnt.argmax()])

In [ ]:
def MultiProcessingProgram(processNum):
  with concurrent.futures.ProcessPoolExecutor() as executor:
    chromo = np.zeros((processNum,N))
    results = [executor.submit(RunProgram, int(math.pow(2,N-1)), N, W, weights, profits) for _ in range(processNum)]
    for index, f in enumerate(concurrent.futures.as_completed(results)):
      chromo[index] = f.result()
  unq, cnt = np.unique(chromo, axis=0, return_counts=True)
  print(unq[cnt.argmax()])

In [ ]:
start_time = time.time()
MultiProcessingProgram(6)
print("--- %s seconds ---" % (time.time() - start_time))

Number of generations:  567
Number of generations:  567
Number of generations:  514
Number of generations:  514
Number of generations:  533
Number of generations:  533
[1. 1. 1. 1. 0. 1. 0. 0. 0. 0.]
--- 62.68116807937622 seconds ---


In [ ]:
start_time = time.time()
print(RunProgram(int(math.pow(2,N-1)), N, W, weights, profits))
print("--- %s seconds ---" % (time.time() - start_time))

Number of generations:  567
[1. 1. 1. 1. 0. 1. 0. 0. 0. 0.]
--- 16.40669894218445 seconds ---


In [ ]:
7
170
442 525 511 593 546 564 617
41 50 49 59 55 57 60
[0. 1. 0. 1. 0. 0. 1.]

In [ ]:
[1 0 1 0 1 0 1 1 1 0 0 0 0 1 1]